## Libraries

In [186]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import log
from causalinference import CausalModel

## Data

In [187]:
file_path = 'resources\Desktoppanel.csv'
df = pd.read_csv(file_path)

In [188]:
df = df[df['year'] > 2014]

keys = df.keys()

na_10percentile = []
na_25percentile = []
na_30percentile = []
na_over_30percentile = []

In [189]:
for key in keys:
    size = df[key].size
    naCount = 0
    for element in df[key]: 
        if pd.isna(element):
            naCount += 1
    naPercentile = naCount / size * 100
    
    if naPercentile < 10:
        na_10percentile.append(key)
    elif naPercentile < 25:
        na_25percentile.append(key)
    elif naPercentile < 30:
        na_30percentile.append(key)
    else:
        na_over_30percentile.append(key)

In [190]:
# df['dq1008'].value_counts()

In [191]:
def check_percentile(key: str):
    if key in na_10percentile:
        print(f'{key} is in 10 percentile | {df[key].isna().sum()} / {df[key].size} ({df[key].isna().sum() / df[key].size * 100:.2f}%)')
    elif key in na_25percentile:
        print(f'{key} is in 25 percentile | {df[key].isna().sum()} / {df[key].size} ({df[key].isna().sum() / df[key].size * 100:.2f}%)')
    elif key in na_30percentile:
        print(f'{key} is in 30 percentile | {df[key].isna().sum()} / {df[key].size} ({df[key].isna().sum() / df[key].size * 100:.2f}%)')
    else:
        print(f'{key} is in over 30 percentile | {df[key].isna().sum()} / {df[key].size} ({df[key].isna().sum() / df[key].size * 100:.2f}%)')


## Data Preprocessing

In [192]:
na_replacement = -9999

회사법인_구분 = 'aq2002'
상장여부 = 'aq2003'
전문경영인_유무 = 'aq2004'
경영형태 = 'aq2901'


주식회사 = 1
유한회사 = 2
합자회사 = 3
합명회사 = 4


예 = 1
아니오 = 2


소유경영체제 = 1
소유주_중심의_경영체제 = 2
주요_경영문제_결정권을_소유주가_가진_경영체제 = 3
전문경영체제 = 4
해당없음 = 97


미상장 = 4


기업체_변화 = ['aq2012r4', 'aq2012r5', 'aq2012r6', 'aq2012r7', 'aq2012r8', 'aq2012r11', 'aq2012r13', 'aq2012r14']


경영_형태 = 'aq2901'
사업체_업력 = 'w_age'
주력_제품의_국내_시장_경쟁_정도 = 'aq3008'
사업장_혁신_유형 = 'aq3901'
능력_근무성적_태도_등에_대해_평가_실시함 = 'cq1001'
성과배분제도_운영_여부 = 'cq3001'
우리사주제도_실시_여부 = 'cq3008'
지난_2년_동안_직무분석_실시한_적_있음 = 'dq1007b'
사무직_근로자의_초과근로가_정기적으로_이루어지는_정도 = 'dq3023'
선택적_근무시간제를_운영함 = 'dq3025'
탄력적_근로시간제를_운영함 = 'dq3027'
작년말_기준_노동조합이_있음 = 'mq1001'
작년_말_기준_노사협의회가_있음 = 'nq1004'


당기_매출액 = 'fpq2001'
전체_근로자_수 = 'epq3008'
# 주_근로시간_1인당_2013까지 = 'dq3919' 
# 주_초과_근로시간_1인당_2013까지 = 'dq3920' 
주_근로시간_1인당_2015이후 = 'per_week' 
주_근로시간_1인당 = 주_근로시간_1인당_2015이후


def determine_management_system(row):
    if row[전문경영인_유무] == 예:
        return 전문경영체제
    elif row[상장여부] < 미상장:
        return 소유주_중심의_경영체제
    elif row[회사법인_구분] == 합자회사:
        return 주요_경영문제_결정권을_소유주가_가진_경영체제
    else:
        return 소유경영체제
    

# X
Independent_var_keys = [
    경영_형태, 사업체_업력, *기업체_변화, 주력_제품의_국내_시장_경쟁_정도, 사업장_혁신_유형, 능력_근무성적_태도_등에_대해_평가_실시함,
    성과배분제도_운영_여부, 우리사주제도_실시_여부, 지난_2년_동안_직무분석_실시한_적_있음, 사무직_근로자의_초과근로가_정기적으로_이루어지는_정도,
    선택적_근무시간제를_운영함, 탄력적_근로시간제를_운영함, 작년말_기준_노동조합이_있음, 작년_말_기준_노사협의회가_있음
]

# y
개인당_노동생산성_변화량 = 'y' 
    
    
df[주_근로시간_1인당] = df[주_근로시간_1인당].fillna(na_replacement)
df[당기_매출액] = df[당기_매출액].fillna(na_replacement)
df[개인당_노동생산성_변화량] = df[당기_매출액] / (df[전체_근로자_수] * df[주_근로시간_1인당])
df[개인당_노동생산성_변화량] = log(df[개인당_노동생산성_변화량].apply(lambda x: x if x > 0 else (999999 if x == 0 else -1 / x)))

fixed_management_data = df[[전문경영인_유무, 상장여부, 회사법인_구분]].apply(determine_management_system, axis=1).fillna(해당없음)
df[경영_형태] = df[경영형태].combine_first(fixed_management_data)

pre_X = df[[*Independent_var_keys]]
df[[*Independent_var_keys]] = pre_X.fillna(na_replacement)

In [193]:
for key_name in 기업체_변화:
    check_percentile(key_name)
    
check_percentile(경영_형태)
check_percentile(사업체_업력)
check_percentile(주력_제품의_국내_시장_경쟁_정도)
check_percentile(사업장_혁신_유형)
check_percentile(능력_근무성적_태도_등에_대해_평가_실시함)
check_percentile(성과배분제도_운영_여부)
check_percentile(우리사주제도_실시_여부)
check_percentile(지난_2년_동안_직무분석_실시한_적_있음)
check_percentile(사무직_근로자의_초과근로가_정기적으로_이루어지는_정도)
check_percentile(선택적_근무시간제를_운영함)
check_percentile(탄력적_근로시간제를_운영함)
check_percentile(작년말_기준_노동조합이_있음)
check_percentile(작년_말_기준_노사협의회가_있음)
check_percentile(당기_매출액)
check_percentile(전체_근로자_수)
check_percentile(주_근로시간_1인당)
check_percentile(개인당_노동생산성_변화량)

aq2012r4 is in 10 percentile | 0 / 11670 (0.00%)
aq2012r5 is in 10 percentile | 0 / 11670 (0.00%)
aq2012r6 is in 10 percentile | 0 / 11670 (0.00%)
aq2012r7 is in 10 percentile | 0 / 11670 (0.00%)
aq2012r8 is in 10 percentile | 0 / 11670 (0.00%)
aq2012r11 is in 10 percentile | 0 / 11670 (0.00%)
aq2012r13 is in 10 percentile | 0 / 11670 (0.00%)
aq2012r14 is in 10 percentile | 0 / 11670 (0.00%)
aq2901 is in over 30 percentile | 0 / 11670 (0.00%)
w_age is in 10 percentile | 0 / 11670 (0.00%)
aq3008 is in 10 percentile | 0 / 11670 (0.00%)
aq3901 is in over 30 percentile | 0 / 11670 (0.00%)
cq1001 is in 10 percentile | 0 / 11670 (0.00%)
cq3001 is in 10 percentile | 0 / 11670 (0.00%)
cq3008 is in 10 percentile | 0 / 11670 (0.00%)
dq1007b is in 10 percentile | 0 / 11670 (0.00%)
dq3023 is in 25 percentile | 0 / 11670 (0.00%)
dq3025 is in 10 percentile | 0 / 11670 (0.00%)
dq3027 is in 10 percentile | 0 / 11670 (0.00%)
mq1001 is in 10 percentile | 0 / 11670 (0.00%)
nq1004 is in 30 percentile | 0 

## Estimation Structure

$$
\ln{Y_{iprt}} = \beta (Treat_i \times Post_t) + X_{iprt}\gamma + Fixed + \varepsilon_{iprt}
$$

### Controlled Vars

|             통제변수              |              코드              |                             비고                             |
|:-----------------------------:|:----------------------------:|:----------------------------------------------------------:|
|            재택근무 비율            |  'epq2007', 'epq9904' or ''  |         (전체 - 재택 수) / 전체 [~2013] or 시행 여부 [2015~]          |
|          최저임금 근로자 비율          |                              |                                                            |
|            기업 순이익             |                              |                                                            |
|         추가 근로제도 시행 여부         |                              |                                                            |
|             경영 형태             |           'aq2901'           | 회사법인_구분 = 'aq2002', 상장여부, 'aq2003', 전문경영인_유무 'aq2004'로 전처리 |
|            사업체 업력             |           'w_age'            |                                                            |
|            기업체 변화             | 'aq2012r4~r8, r11, r13, r14' |                                                            |
|      주력 제품의 국내 시장 경쟁 정도       |           'aq3008'           |                                                            |
|           사업장 혁신 유형           |           'aq3901'           |                                                            |
|   능력, 근무성적, 태도 등에 대해 평가 실시함   |           'cq1001'           |                                                            |
|         성과배분제도 운영 여부          |           'cq3001'           |                                                            |
|         우리사주제도 실시 여부          |           'cq3008'           |                                                            |
|    지난 2년 동안 직무분석 실시한 적 있음     |          'dq1007b'           |                                                            |
| 사무직 근로자의 초과근로가 정기적으로 이루어지는 정도 |           'dq3023'           |                      na at 2005, 2007                      |
|        선택적 근무시간제를 운영함         |           'dq3025'           |                                                            |
|        탄력적 근로시간제를 운영함         |           'dq3027'           |                                                            |
|        작년말 기준 노동조합이 있음        |           'mq1001'           |                                                            |
|       작년 말 기준 노사협의회가 있음       |           'nq1004'           |                    mq1001 + nq1004 가능할듯                    | 

### Fixed vars
 
|    고정효과    |
|:----------:|
|     지역     |
|     연도     |
|    산업분류    |
|     규모     |
| 산업규모 * 연도  |
|   연도별 기업   |
|   연도별 지역   |

## Regression

In [194]:
# df = df.set_index(['id', 'year'])
# 
# y = df[개인당_노동생산성_변화량]
# X = df[[*Independent_var_keys]]
# 
# X = sm.add_constant(X)
# 
# model = PanelOLS(y, X, entity_effects=True, time_effects=True)
# 
# result = model.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
# 
# result

C:\Users\3doll\miniconda3\envs\untitled\lib\site-packages\linearmodels\panel\results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,y,R-squared:,0.0045
Estimator:,PanelOLS,R-squared (Between):,0.0524
No. Observations:,11670,R-squared (Within):,0.0165
Date:,"Mon, Jun 10 2024",R-squared (Overall):,0.0409
Time:,18:54:50,Log-likelihood,-1.837e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.8438
Entities:,3571,P-value,0.0123
Avg Obs:,3.2680,Distribution:,"F(20,8076)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,2.0748


In [195]:
운수업 = 'H'

df['Treat'] = np.where((df['year'] >= 2019) & (df['ind'] == 운수업), 1, 0)

Y = df[개인당_노동생산성_변화량].values
D = df['Treat'].values
X = df[[*Independent_var_keys]].values

causal = CausalModel(Y, D, X)
# causal.est_propensity_s()
causal.est_via_matching(bias_adj=True)

causal.estimates

KeyError: 'year'

## Visualization

In [ ]:
coefficients = result.params.drop('const') * 100

plt.figure(figsize=(10, 5))
coefficients.plot(kind='bar')
plt.title('Regression Coefficients')
plt.grid(True)
plt.show()